In [1]:
!pip install pygame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.8 MB/s eta 0:00:0000:0100:01


In [4]:
import pygame
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim

# Define the colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Initialize Pygame
pygame.init()

# Set the screen dimensions
SCREEN_WIDTH = 1200
SCREEN_HEIGHT = 800
screen = pygame.display.set_mode([SCREEN_WIDTH, SCREEN_HEIGHT])

# set the title of the window
pygame.display.set_caption("Swarm Simulation")

# set the clock for the game loop
clock = pygame.time.Clock()

# agent class
class Agent(pygame.sprite.Sprite):
    def __init__(self, x, y, color):
        super().__init__()

        # Define the image of the agent
        self.image = pygame.Surface([10, 10])
        self.image.fill(color)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

        # initialize the velocity and speed of the agent
        self.velocity = torch.tensor([random.uniform(-1, 1), random.uniform(-1, 1)], requires_grad=True)
        self.speed = 4

    def update(self, agents, model):
        # Compute the alignment, cohesion, and separation forces using the PyTorch model
        alignment_force, cohesion_force, separation_force = model(self, agents)

        # Combine the forces
        force = torch.zeros(2)
        force[0] = alignment_force[0] + cohesion_force[0] + separation_force[0]
        force[1] = alignment_force[1] + cohesion_force[1] + separation_force[1]

        # Update the velocity 
        self.velocity = self.velocity + force

        # Update the position of the speed
        speed = torch.norm(self.velocity)

        if speed > self.speed:
            self.velocity = (self.velocity/speed) * self.speed

        # Update the position of the agent
        self.rect.x += self.velocity[0].detach().item()
        self.rect.y += self.velocity[1].detach().item()

        # Check if the agent is out of bounds
        if self.rect.x < 0:
            self.rect.x = SCREEN_WIDTH
        elif self.rect.x > SCREEN_WIDTH:
            self.rect.x = 0
        if self.rect.y < 0:
            self.rect.y = SCREEN_HEIGHT
        elif self.rect.y > SCREEN_HEIGHT:
            self.rect.y = 0
    def distance(self, agent):
        return math.sqrt((self.rect.center[0] - agent.rect.center[0])**2 + (self.rect.center[1] - agent.rect.center[1])**2)



def distance_v2(agent, agent_temp):
    return math.sqrt((agent[0] - agent_temp[0])**2 + (agent[1] - agent_temp[1])**2)

# model for the swarm behavior
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.alignment_layer = nn.Linear(4, 2)
        self.cohesion_layer = nn.Linear(4, 2)
        self.separation_layer = nn.Linear(4, 2)

    def forward(self, agent, agents):

        # Compute the alignment force using the average velocity of the neighbors
        neighbor_velocities = [agent_temp.velocity for agent_temp in agents if agent != agent_temp and agent.distance(agent_temp) < 50]


        if len(neighbor_velocities) == 0:
            alignment_force = torch.zeros(2)
        else:
            avg_velocity = torch.mean(torch.stack(neighbor_velocities), dim=0)
            # print('avg_velocity', avg_velocity)
            alignment_input = torch.cat((agent.velocity, avg_velocity))
            # print('alignment_input', alignment_input)
            alignment_force = self.alignment_layer(alignment_input)

        # Compute the cohesion force using the center of mass of the neighbors
        neighbor_positions = [agent_temp.rect.center for agent_temp in agents if agent != agent_temp and agent.distance(agent_temp) < 50]

        if len(neighbor_positions) == 0:
            cohesion_force = torch.zeros(2)
        else:
            center_of_mass = torch.mean(torch.tensor(neighbor_positions).float(), dim=0)
            # print('center_of_mass', center_of_mass)
            # print('agent.rect.center', agent.rect.center)
            # cohesion_input = torch.cat((agent.rect.center, center_of_mass))
            cohesion_input = torch.cat((torch.tensor([agent.rect.center[0], agent.rect.center[1]]), center_of_mass))
            # print('cohesion_input', cohesion_input)
            cohesion_force = self.cohesion_layer(cohesion_input)

        # Compute theseparation force using the repulsion force from the neighbors
        neighbor_positions = [agent_temp.rect.center for agent_temp in agents if agent != agent_temp and agent.distance(agent_temp) < 25]

        if len(neighbor_positions) == 0:
            separation_force = torch.zeros(2)
        else:
            repulsion_force = torch.zeros(2)
            for neighbor_position in neighbor_positions:
                # print('neighbor_position', torch.tensor(neighbor_position))
                # print('agent.rect.center', torch.tensor(agent.rect.center))
                distance = distance_v2(torch.tensor(agent.rect.center), torch.tensor(neighbor_position))
                # distance = agent.distance(neighbor_position.rect.center)
                if distance == 0:
                    distance = 0.0001
                repulsion_force += (torch.tensor(agent.rect.center) - torch.tensor(neighbor_position))/distance**2
            separation_force = self.separation_layer(torch.cat((agent.velocity, repulsion_force)))

        return alignment_force, cohesion_force, separation_force

# NUMBER OF EPOCHS
EPOCHS = 400

# Train the PyTorch model using the agents' behavior
def train_model(model, agents):
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)
    for epoch in range(EPOCHS):
        running_loss = 0.0
        for agent in agents:
            alignment_force, cohesion_force, separation_force = model(agent, agents)
            target = torch.zeros(alignment_force.shape, requires_grad=True)
            loss = criterion(alignment_force + cohesion_force + separation_force, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print('Epoch %d loss: %.3f' % (epoch + 1, running_loss/len(agents)))

# Create the agents in pygame environment
agents = pygame.sprite.Group()

# number of agents
NUMBER_OF_AGENTS = 150

for i in range(NUMBER_OF_AGENTS):
    agent = Agent(random.randint(0, SCREEN_WIDTH), random.randint(0, SCREEN_HEIGHT), GREEN)
    agents.add(agent)

# get the model for the swarm behaviour
model = Model()

print(model)

# Train the model
train_model(model, agents)

# Set the game loop
done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True

    # Update the agents using the model
    for agent in agents:
        agent.update(agents, model)

    # Draw the agents
    screen.fill(BLACK)
    agents.draw(screen)
    pygame.display.flip()
    clock.tick(60)

pygame.quit()




shm_open() failed: No such file or directory
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5703:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


SwarmModel(
  (alignment_layer): Linear(in_features=4, out_features=2, bias=True)
  (cohesion_layer): Linear(in_features=4, out_features=2, bias=True)
  (separation_layer): Linear(in_features=4, out_features=2, bias=True)
)
Epoch 1 loss: 8717.967
Epoch 2 loss: 808.739
Epoch 3 loss: 55.313
Epoch 4 loss: 30.944
Epoch 5 loss: 30.613
Epoch 6 loss: 30.707
Epoch 7 loss: 30.781
Epoch 8 loss: 30.815
Epoch 9 loss: 30.811
Epoch 10 loss: 30.775
Epoch 11 loss: 30.710
Epoch 12 loss: 30.621
Epoch 13 loss: 30.510
Epoch 14 loss: 30.381
Epoch 15 loss: 30.237
Epoch 16 loss: 30.080
Epoch 17 loss: 29.913
Epoch 18 loss: 29.736
Epoch 19 loss: 29.552
Epoch 20 loss: 29.362
Epoch 21 loss: 29.167
Epoch 22 loss: 28.968
Epoch 23 loss: 28.766
Epoch 24 loss: 28.561
Epoch 25 loss: 28.355
Epoch 26 loss: 28.147
Epoch 27 loss: 27.938
Epoch 28 loss: 27.729
Epoch 29 loss: 27.519
Epoch 30 loss: 27.310
Epoch 31 loss: 27.100
Epoch 32 loss: 26.891
Epoch 33 loss: 26.682
Epoch 34 loss: 26.473
Epoch 35 loss: 26.265
Epoch 36 los